**UCS Search**

Implementation with Visualization of (6x10) matrix at each step. Shows how search explores and finds the goal get the path. 

**Note:** If you want just the final answer then its at the last cell

In [3]:
import pandas as pd
import numpy as np
from IPython.display import Markdown, display
import heapdict

# creating dictionary that stores info abour each cell of the cliff walker maze as per the question
maze_dict=dict()


start_position = (6,1)
goal_position = (6,10)
fringe_ucs=heapdict.heapdict() # heap_dictionary that works like a priority queue for implementing ucs
path_ucs=[] # returned path for ucs

# backtracking_dict dictionary will be later used to back track the path of traversal that actually led to finding goal
backtracking_dict={start_position:[]}

#DeathZone in red as per question
#Shallow region in white as per question
#Deep region in green as per question
def init_cliff_walker(start_pos, goal_pos):
  for i in range(1,7):
    for j in range(1,11):
      if i==6 and (j<=9 and j>=2):
        maze_dict[(i,j)] = [" ","Deathzone","unvisited"]
      elif (i>=3 and i<=5) and ((j<=4 and j>=2) or (j<=8 and j>=6)):
        maze_dict[(i,j)] = [" ","Deep","unvisited"]
      else:
        maze_dict[(i,j)] = [" ","Shallow","unvisited"]
  maze_dict[start_pos][0] = "START"
  maze_dict[goal_pos][0] = "GOAL"

def print_cliff_walker():
  row_lst = []
  for row_num in range(1,7):
    lst=[]
    for col_num in range(1,11):
      if (row_num,col_num) == start_position or (row_num,col_num) == goal_position or ((row_num,col_num) in path_ucs):
        lst.append(str(maze_dict[(row_num,col_num)][0]))
      elif maze_dict[(row_num,col_num)][0] != " ":
        lst.append(maze_dict[(row_num,col_num)][0])
      elif maze_dict[(row_num,col_num)][2] == "unvisited":
        lst.append(5*'_')
    row_lst.append(lst)
  
  idx_lst=[str(i) for i in range(1,7)]
  col_lst=[str(i) for i in range(1,11)]

  data_arr = np.array(row_lst)
  df = pd.DataFrame(data_arr, index=idx_lst, columns=col_lst)
  df = df.style.apply(style_specific_cell, axis=None)
  return df

def style_specific_cell(x):

    font_typ = 'font-weight: bold; text-align: center;'
    bg_color = ''
    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
    for key in maze_dict.keys():
      df1.iloc[key[0]-1, key[1]-1] = font_typ
      if maze_dict[key][1] == "Deep":
        bg_color = 'background-color: lightgreen;'
      elif maze_dict[key][1] == "Deathzone":
        bg_color = 'background-color: red;'
      elif maze_dict[key][1] == "Shallow":
        bg_color = 'background-color: white;'
      df1.iloc[key[0]-1, key[1]-1] += bg_color

      if key in path_ucs:
        df1.iloc[key[0]-1, key[1]-1] += 'color: blue;'
      else:
        df1.iloc[key[0]-1, key[1]-1] += 'color: black;'
        
    return df1

def update_map(updated_cost, update_location):
  if update_location == start_position:
    maze_dict[update_location][0] = "START="+ str(updated_cost)
  elif update_location == goal_position:
    maze_dict[update_location][0] = "GOAL="+ str(updated_cost)
  else:
    maze_dict[update_location][0] = str(updated_cost)

def get_cell_expense(cell_type):
  if cell_type == "Deep":
    return 5
  elif cell_type == "Shallow":
    return 1

# checks for valid neighbors/children to explore as per environment conditions. 
# Here valid means maze boundary condtions to be in 6x10 matrix, not encountering red deathzone and not revisiting already visited node
def get_valid_neighbors(curr_pos):
  neighbors = [(curr_pos[0]-1,curr_pos[1]), (curr_pos[0],curr_pos[1]-1), (curr_pos[0],curr_pos[1]+1), (curr_pos[0]+1,curr_pos[1])]
  valid=[]
  for i in neighbors:
    if(i[0]<7 and i[0]>0) and (i[1]>0 and i[1]<11) and maze_dict[i][2] == "unvisited" and maze_dict[i][1]!="Deathzone" and (maze_dict[i][0] == " " or maze_dict[i][0] == "GOAL"):
      valid.append(i)
  return valid

  
init_cliff_walker(start_position, goal_position)
print("Initiating empty maze for UCS")
display(print_cliff_walker())

print("***----------------------------------------------------------------------------------------------------------------------------------------------***")
print("now starting UCS")


def ucs():

  curr,gn_cost = fringe_ucs.popitem()  
  
  # ignores visited positions to avoid redundant operation
  while maze_dict[curr][2] == "visited":
    curr,gn_cost = fringe_ucs.popitem() # popped item is (location, cost)
  
  maze_dict[curr][2] = "visited"

  print("popped "+str(curr)+" and tracked below:")
  
  if curr == goal_position:
    global ucs_path_cost # records path cost sum when GOAL reached 
    update_map(gn_cost, curr)
    path_ucs.append(curr)
    backtracking_dict[tuple()]= [curr]
    ucs_path_cost=gn_cost
    return True
  
  #collecting valid neighbors to expand the UCS tree
  children = get_valid_neighbors(curr)
  # adding new children/neighbors to fringe (priority queue)
  for child in children:
    new_cost = get_cell_expense(maze_dict[child][1]) + gn_cost
    fringe_ucs[child] = new_cost
    update_map(new_cost, child)
    

  #updating backtracking dictionary
  for child in children:
    if child not in backtracking_dict:
      backtracking_dict[child] = [curr]
    
  print("Fringe => "+ str(list(fringe_ucs.items())))
  display(print_cliff_walker())
  # continues recursive call till fringe empty or until GOAL not found
  while (fringe_ucs.items())!=[]:  
    result = ucs()
    if result == True:
      return True  
  return False


#adding root/start node to ucs-fringe to start ucs and updating map to print its cost as well
fringe_ucs[start_position] = get_cell_expense(maze_dict[start_position][1])
update_map(get_cell_expense(maze_dict[start_position][1]), start_position)
print("Fringe => "+ str(list(fringe_ucs.items())))

if ucs():
  print("UCS found GOAL")
else:
  print("UCS failed to find GOAL")

#using backtracking to get path
temp_key=(6,10)
while backtracking_dict[temp_key] != []:
  ans = backtracking_dict[temp_key][0]
  path_ucs.append(ans)
  temp_key = ans
path_ucs = [path_ucs[x] for x in range( len(path_ucs)-1,-1,-1)] # reversing list to go from START to GOAL
print("UCS path backtracked START to GOAL =>"+str(path_ucs))

print("UCS Path Cost = "+str(ucs_path_cost))
print()
print("Output of explored areas with output cost at each step higlighted in blue color:")
display(print_cliff_walker())

Initiating empty maze for UCS


,1,2,3,4,5,6,7,8,9,10
1,_____,_____,_____,_____,_____,_____,_____,_____,_____,_____
2,_____,_____,_____,_____,_____,_____,_____,_____,_____,_____
3,_____,_____,_____,_____,_____,_____,_____,_____,_____,_____
4,_____,_____,_____,_____,_____,_____,_____,_____,_____,_____
5,_____,_____,_____,_____,_____,_____,_____,_____,_____,_____
6,START,_____,_____,_____,_____,_____,_____,_____,_____,GOAL


***----------------------------------------------------------------------------------------------------------------------------------------------***
now starting UCS
Fringe => [((6, 1), 1)]
popped (6, 1) and tracked below:
Fringe => [((5, 1), 2)]


,1,2,3,4,5,6,7,8,9,10
1,_____,_____,_____,_____,_____,_____,_____,_____,_____,_____
2,_____,_____,_____,_____,_____,_____,_____,_____,_____,_____
3,_____,_____,_____,_____,_____,_____,_____,_____,_____,_____
4,_____,_____,_____,_____,_____,_____,_____,_____,_____,_____
5,2,_____,_____,_____,_____,_____,_____,_____,_____,_____
6,START=1,_____,_____,_____,_____,_____,_____,_____,_____,GOAL


popped (5, 1) and tracked below:
Fringe => [((4, 1), 3), ((5, 2), 7)]


...

,1,2,3,4,5,6,7,8,9,10
1,6,7,8,9,10,11,12,13,14,15
2,5,6,7,8,9,10,11,12,13,14
3,4,9,12,13,10,15,16,17,14,15
4,3,8,13,16,11,16,21,20,15,16
5,2,7,12,17,12,17,22,21,16,17
6,START=1,_____,_____,_____,_____,_____,_____,_____,_____,GOAL=18


popped (3, 8) and tracked below:
Fringe => [((4, 8), 20), ((4, 7), 21), ((5, 8), 21), ((5, 7), 22), ((6, 10), 18)]


,1,2,3,4,5,6,7,8,9,10
1,6,7,8,9,10,11,12,13,14,15
2,5,6,7,8,9,10,11,12,13,14
3,4,9,12,13,10,15,16,17,14,15
4,3,8,13,16,11,16,21,20,15,16
5,2,7,12,17,12,17,22,21,16,17
6,START=1,_____,_____,_____,_____,_____,_____,_____,_____,GOAL=18


popped (6, 10) and tracked below:
UCS found GOAL
UCS path backtracked START to GOAL =>[(6, 1), (5, 1), (4, 1), (3, 1), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (3, 9), (4, 9), (5, 9), (5, 10), (6, 10)]
UCS Path Cost = 18

Output of explored areas with output cost at each step higlighted in blue color:


,1,2,3,4,5,6,7,8,9,10
1,6,7,8,9,10,11,12,13,14,15
2,5,6,7,8,9,10,11,12,13,14
3,4,9,12,13,10,15,16,17,14,15
4,3,8,13,16,11,16,21,20,15,16
5,2,7,12,17,12,17,22,21,16,17
6,START=1,_____,_____,_____,_____,_____,_____,_____,_____,GOAL=18


**Final answer of UCS below**

In [4]:
print("UCS path backtracked START to GOAL =>"+str(path_ucs))
print("UCS Path Cost = "+str(ucs_path_cost))
print()
print("Output of explored areas with output cost at each step higlighted in blue color:")
display(print_cliff_walker())

UCS path backtracked START to GOAL =>[(6, 1), (5, 1), (4, 1), (3, 1), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (3, 9), (4, 9), (5, 9), (5, 10), (6, 10)]
UCS Path Cost = 18

Output of explored areas with output cost at each step higlighted in blue color:


,1,2,3,4,5,6,7,8,9,10
1,6,7,8,9,10,11,12,13,14,15
2,5,6,7,8,9,10,11,12,13,14
3,4,9,12,13,10,15,16,17,14,15
4,3,8,13,16,11,16,21,20,15,16
5,2,7,12,17,12,17,22,21,16,17
6,START=1,_____,_____,_____,_____,_____,_____,_____,_____,GOAL=18
